In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Первичный анализ данных

In [261]:
df = pd.read_csv('dano_dataset_travel.csv', delimiter=';', decimal=',', encoding='utf-8', low_memory=False)
print(df.head())

   order_online_payment_flg  \
0                       1.0   
1                       1.0   
2                       1.0   
3                       1.0   
4                       1.0   

                                          account_rk  \
0  209c833731e84d21b5b7e673e0fb848749b9e7d29cda64...   
1  2bbcde706bead3731f2dc8dfbeefb4e12b42ac63e3d8ba...   
2  a82919af3a5a1869f9becdcffa5c7d303d105797ce45b8...   
3  9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...   
4  3133e7ae6698dcda9754d6a8b449782320260ff1259547...   

                                           client_rk  \
0  f4959ffb27271192727050953ecb27a8a8a38af413f3d8...   
1  7025587bc277176246bc44dff396036552a41d5a92d6aa...   
2  0368b36ccd204d631305233a1f952bc5fb0e3fd9f690fa...   
3  1346344779d7bd788d03ec2ad1908daf71c6358aca47f1...   
4  1a2a4351a96099f9e49bbe2fc7236d41ac63b444176058...   

                                            order_rk loyalty_program_type_nm  \
0  fe1c6ce13774d102c655df4c01d54c34d495ef3d4c7e63...       

In [262]:
print(f"Размер данных: {df.shape}")

Размер данных: (835938, 56)


In [263]:
print("\nИнформация о столбцах:")
print(df.info())


Информация о столбцах:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835938 entries, 0 to 835937
Data columns (total 56 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   order_online_payment_flg     786885 non-null  float64
 1   account_rk                   835938 non-null  object 
 2   client_rk                    835938 non-null  object 
 3   order_rk                     835938 non-null  object 
 4   loyalty_program_type_nm      754957 non-null  object 
 5   bundle_nm                    588049 non-null  object 
 6   order_type_cd                786885 non-null  object 
 7   order_status_cd              779514 non-null  object 
 8   party_first_order_dt         786885 non-null  object 
 9   party_first_order_type_dt    786767 non-null  object 
 10  free_cancel_booking_dttm     75600 non-null   object 
 11  created_dttm                 786885 non-null  object 
 12  cancel_dttm                  0 non

In [264]:
missing_data = df.isnull().sum()
missing_percent = ((missing_data / len(df)) * 100).round(2)

missing_df = pd.DataFrame({
	'Количество пропусков' : missing_data,
	'Процент пропусков' : missing_percent
})

missing_df = missing_df.sort_values(by = 'Процент пропусков', ascending=False)

In [265]:
print('\nПропущенные данные:')
print(missing_df[missing_df['Количество пропусков'] > 0])


Пропущенные данные:
                             Количество пропусков  Процент пропусков
cancel_dttm                                835938             100.00
call_contact_1m_flg                        835667              99.97
call_contact_3m_flg                        835667              99.97
call_contact_6m_flg                        835667              99.97
good_email_address_flg                     816140              97.63
bad_email_address_flg                      816140              97.63
last_email_send_dt                         816140              97.63
bounce_cd                                  796263              95.25
free_cancel_booking_dttm                   760338              90.96
hotel_country                              673690              80.59
hotel_city                                 673690              80.59
book_end_dttm                              664390              79.48
education_level_cd                         360834              43.17
loyalty_accru

Столбец cancel_dttm - полностью пуст.
13 столбцов имеет от 90% до 99.97% пропусков.  

In [266]:
duplicates = df.duplicated().sum()
print(f"\nПолных дубликатов строк: {duplicates}")


Полных дубликатов строк: 0


Полных дубликатов строк нет, это значит, каждая запись уникальна. 

In [267]:
if 'order_rk' in df.columns:
    duplicate_orders = df['order_rk'].duplicated().sum()
    print(f"Дубликатов order_rk: {duplicate_orders}")

Дубликатов order_rk: 49052


Дубликаты в order_rk (идентификатор заказа) могут означать, что некоторые заказы учитываются несколько раз. Например из-за изменения order_status_cd (статус заказа). 

In [268]:
df['order_type_cd']

0         AIR
1         HOT
2         AIR
3         HOT
4         AIR
         ... 
835933    AIR
835934    AIR
835935    AIR
835936    AIR
835937    AIR
Name: order_type_cd, Length: 835938, dtype: object

In [269]:
print("Типы заказов:")
print(df['order_type_cd'].value_counts(dropna=False))

Типы заказов:
order_type_cd
AIR    615337
HOT    171548
NaN     49053
Name: count, dtype: int64


In [270]:
# Расчет распределения типов заказов в процентах
order_type_distribution = df['order_type_cd'].value_counts(dropna=False, normalize=True) * 100

print("Типы заказов (в процентах):")
print(order_type_distribution.round(2).astype(str) + '%')

Типы заказов (в процентах):
order_type_cd
AIR    73.61%
HOT    20.52%
NaN     5.87%
Name: proportion, dtype: object


Большинство заказов - авиабилеты. Почти 6% данных не имеют указанного типа. 

In [271]:
print('\nСтатусы заказов:')
print(df['order_status_cd'].value_counts(dropna = False))


Статусы заказов:
order_status_cd
SUC    759950
NaN     56424
PRC     18467
ERR       650
NSH       304
CNC        84
REJ        58
BRN         1
Name: count, dtype: int64


In [272]:
print('\nСтатусы заказов (в процентах):')
status_distribution = df['order_status_cd'].value_counts(dropna=False, normalize=True) * 100
print(status_distribution.round(2).astype(str) + '%')



Статусы заказов (в процентах):
order_status_cd
SUC    90.91%
NaN     6.75%
PRC     2.21%
ERR     0.08%
NSH     0.04%
CNC     0.01%
REJ     0.01%
BRN      0.0%
Name: proportion, dtype: object


Подавляющее большинство заказов имеют статус SUC (успешно завершен), составляя около 90.91%. При этом 73.61% приходится на авиабилеты, а 20.52% - на отели. 

In [273]:
hotels_df = df[df['order_type_cd'] == 'HOT']

In [274]:
airport_df = df[df['order_type_cd'] == 'AIR']

In [275]:
print('\nСтатусы бронирования отелей:')
print(hotels_df['order_status_cd'].value_counts(dropna=False))


Статусы бронирования отелей:
order_status_cd
SUC    163794
NaN      7371
NSH       304
REJ        58
CNC        21
Name: count, dtype: int64


In [276]:
print('\nСтатусы бронирования авиабилетов:')
print(airport_df['order_status_cd'].value_counts(dropna=False))


Статусы бронирования авиабилетов:
order_status_cd
SUC    596156
PRC     18467
ERR       650
CNC        63
BRN         1
Name: count, dtype: int64


In [277]:
successful_hotels = hotels_df['order_status_cd'] == 'SUC'
print(f"\nУспешных бронирований отелей: {successful_hotels.sum()} ({successful_hotels.mean()*100:.1f}%)")


Успешных бронирований отелей: 163794 (95.5%)


In [278]:
successful_airport = airport_df['order_status_cd'] == 'SUC'
print(f"\nУспешных бронирований авиабилетов: {successful_airport.sum()} ({successful_airport.mean()*100:.1f}%)")


Успешных бронирований авиабилетов: 596156 (96.9%)


Процент успешных бронирований довольно высок и по бронированию отелей, и по бронированию авиабилетов. Тем не менее авиабилеты имеют более высокий процент успешных бронирований. 

In [279]:
df['loyalty_accrual_rub_amt']

0            NaN
1          277.0
2          565.0
3            NaN
4          584.0
           ...  
835933     224.0
835934       NaN
835935     157.0
835936    2403.0
835937     191.0
Name: loyalty_accrual_rub_amt, Length: 835938, dtype: float64

In [280]:
df['promo_code_discount_amt']

0         0.0
1         NaN
2         0.0
3         NaN
4         0.0
         ... 
835933    0.0
835934    0.0
835935    0.0
835936    0.0
835937    0.0
Name: promo_code_discount_amt, Length: 835938, dtype: float64

In [281]:
if 'promo_code_discount_amt' in hotels_df.columns:
    with_promo = hotels_df['promo_code_discount_amt'].notna() | (hotels_df['loyalty_accrual_rub_amt'] > 0)

print(f"\nБронирований отелей с промокодом/кэшбэком: {with_promo.sum()} ({with_promo.mean()*100:.2f}%)")


Бронирований отелей с промокодом/кэшбэком: 105273 (61.37%)


In [282]:
organic = ~with_promo
print(f"Органическое бронирование отелей (без стимулирования): {organic.sum()} ({organic.mean()*100:.2f}%)")

Органическое бронирование отелей (без стимулирования): 66275 (38.63%)


61.37% бронирования отелей совершается с применением промокодов или кэшбека, что демонстрирует высокую зависимость спроса от макретинговых стимулов. 

Демографический анализ.

In [284]:
if 'age' in df.columns:
    print("Описательная статистика возраста:")
    print(df['age'].describe().round(2))

Описательная статистика возраста:
count    835937.00
mean         36.07
std           9.37
min          14.00
25%          29.00
50%          35.00
75%          41.00
max          98.00
Name: age, dtype: float64


Средний возраст 36 лет, минимальный возраст 14 лет, максимальный возраст 98 лет. 

In [285]:
if 'age_type_cd' in df.columns:
    age_groups = (df['age_type_cd'].value_counts(normalize=True) * 100).round(2)
    print(age_groups)

age_type_cd
A20    67.28
A40    29.56
A60     1.91
A00     1.25
Name: proportion, dtype: float64


Основная аудитория это молодые клиенты (20 - 40 лет) 67.28% и взрослые клиенты (40 - 60 лет) 29.56%.

In [286]:
if 'gender_cd' in df.columns:
    print("\nРаспределение по полу:")
    gender_dist = (df['gender_cd'].value_counts(normalize=True) * 100).round(2)
    print(gender_dist)


Распределение по полу:
gender_cd
M    62.93
F    37.07
Name: proportion, dtype: float64


Мужчины 62.93%, женщины 37.07%.
Преобладающее число пользователей являются мужчинами. Гендерный дисбаланс (почти 2:1).

In [287]:
if 'marital_status_cd' in df.columns:
    print("\nСемейный статус клиентов:")
    marital_status = (df['marital_status_cd'].value_counts(normalize=True) * 100).round(2)
    print(marital_status)


Семейный статус клиентов:
marital_status_cd
MAR    43.84
UNM    42.38
CIV     6.79
DIV     6.55
WID     0.44
DLW     0.00
Name: proportion, dtype: float64


Женаты/замужем 43.84%, холостые 42.38%
Количество пользователей, которые состоят в браке и не состоят, примерно одинаково.

In [288]:
(pd.crosstab(df['gender_cd'], df['marital_status_cd'], normalize='index')*100).round(2)

marital_status_cd,CIV,DIV,DLW,MAR,UNM,WID
gender_cd,,,,,,
F,7.53,9.06,0.0,38.44,44.11,0.86
M,6.34,5.08,0.0,46.99,41.39,0.19


In [289]:
if 'children_cnt' in df.columns:
    print("\nРаспределение по количеству детей:")
    children_dist = (df['children_cnt'].value_counts(normalize=True) * 100).round(2)
    print(children_dist)


Распределение по количеству детей:
children_cnt
0.0    95.52
1.0     2.40
2.0     1.62
3.0     0.41
4.0     0.04
5.0     0.01
6.0     0.00
8.0     0.00
Name: proportion, dtype: float64


Без детей 95.52%. 
Преобладающее число людей не имеют детей, дети есть у 4.48% пользователей. 

In [290]:
if 'education_level_cd' in df.columns:
    print("\nУровень образования клиентов:")
    education_dist = (df['education_level_cd'].value_counts(normalize=True) * 100).round(2)
    print(education_dist)


Уровень образования клиентов:
education_level_cd
GRD    66.77
UGR    13.47
SCH    12.19
PGR     5.73
ACD     1.83
Name: proportion, dtype: float64


Высшее образование есть у 66.77% пользователей. 

Демографический портрет клиента представляет собой молодого человека в возрасте от 20 до 40 лет, с заметным преобладанием мужской аудитории (62.93 % против 37.07 % женской аудитории). Люди возраста старше 60 лет (1.91%) и молодые люди до 20 лет (1.25%) представлены незначительно. Семьи с одним ребенком и более встречаются редко (всего 4.48 %, из которых лишь у 2.40 % есть один ребенок). Более половины клиентов имеют высшее образование (66.77 %), еще 13.47 % имеют неполное высшее образованиее.

In [291]:
if 'lvn_state_nm' in df.columns:
    print("\nТоп-10 регионов проживания:")
    top_regions = df['lvn_state_nm'].value_counts().head(10)
    print(top_regions)


Топ-10 регионов проживания:
lvn_state_nm
Г МОСКВА              172961
МОСКОВСКАЯ ОБЛ         80060
Г САНКТ-ПЕТЕРБУРГ      70977
СВЕРДЛОВСКАЯ ОБЛ       34714
КРАСНОДАРСКИЙ КРАЙ     26948
РЕСП ТАТАРСТАН         26079
РЕСП БАШКОРТОСТАН      22124
ЧЕЛЯБИНСКАЯ ОБЛ        17377
САМАРСКАЯ ОБЛ          16571
ЛЕНИНГРАДСКАЯ ОБЛ      14064
Name: count, dtype: int64


Большинство пользователей являются жителями Москвы и Московской области. 

In [292]:
if 'avia_dep_city' in df.columns:
    print("\nТоп-10 городов вылета:")
    print(df['avia_dep_city'].value_counts().head(10))


Топ-10 городов вылета:
avia_dep_city
Москва              202016
Санкт-Петербург      55202
Сочи (Адлер)         32054
Екатеринбург         24003
Минеральные Воды     14854
Казань               14110
Уфа                  12722
Стамбул              11285
Новосибирск          10404
Самара               10246
Name: count, dtype: int64


Чаще всего пользователи вылетали из аэропортов Москвы и Санкт-Петербурга. 

In [293]:
if 'avia_arr_city' in df.columns:
    print("\nТоп-10 городов прилета:")
    print(df['avia_arr_city'].value_counts().head(10))


Топ-10 городов прилета:
avia_arr_city
Москва              188962
Санкт-Петербург      54341
Сочи (Адлер)         31048
Екатеринбург         22843
Минеральные Воды     14364
Казань               13547
Душанбе              13524
Уфа                  12480
Стамбул              11158
Самара                9933
Name: count, dtype: int64


Прилетают пользователи тоже чаще всего в аэропорты Москвы и Санкт-Петербурга. 

In [294]:
if 'hotel_country' in df.columns:
    print("\nТоп-10 стран отелей:")
    print(df['hotel_country'].value_counts().head(10))


Топ-10 стран отелей:
hotel_country
Россия       120729
Турция         7511
Таиланд        5611
ОАЭ            3903
Италия         2002
Китай          1912
Япония         1787
Грузия         1519
Казахстан      1322
Индонезия      1204
Name: count, dtype: int64


Чаще всего пользователи посещают города России и Турции. 

In [295]:
if 'hotel_city' in df.columns:
    print("\nТоп-10 городов отелей:")
    print(df['hotel_city'].value_counts().head(10))


Топ-10 городов отелей:
hotel_city
Москва             22624
Санкт-Петербург    13881
Сочи                6098
Казань              3675
Стамбул             3379
Дубай               3111
Екатеринбург        2447
Краснодар           2341
Эсто-Садок          2301
Нижний Новгород     2150
Name: count, dtype: int64


Географический портрет клиентов и их предпочтений. 
Большинство клиентов проживает в Москве (172 961 человек) и Московской области (80 060 человек), третье место занимает Санкт-Петербург (70 977 человек). География вылетов и прилётов демонстрирует следующие тенденции: лидируют Москва (202 016 вылетов и 188 962 прилёта) и Санкт-Петербург (55 202 вылета и 54 341 прилёт), третьим популярным направлением является Сочи (32 054 вылета и 31 048 прилётов). Среди международных направлений выделяется Стамбул (11 285 вылетов и 11 158 прилётов). Что касается внутрироссийского туризма, то предпочтения туристов распределяются следующим образом: наибольший интерес вызывают отели Москвы (22 624 отеля), Санкт-Петербурга (13 881 отель) и Сочи (6 098 отелей). Наиболее востребованными зарубежными направлениями среди забронированных отелей стали Турция (7 511 отелей), Таиланд (5 611 отелей) и Объединённые Арабские Эмираты (3 903 отеля).

In [296]:
avg_order_amount = df['nominal_price_rub_amt'].mean()
print(f"\nСредняя сумма заказа: {avg_order_amount:.2f} руб.")



Средняя сумма заказа: 15178.10 руб.


In [297]:
avg_discount = df['promo_code_discount_amt'].mean()
print(f"Средний размер скидки: {avg_discount:.2f} руб.")

Средний размер скидки: 5.45 руб.


Несмотря на небольшие средние скидки 5.45 рублей ими активно пользуются. 

In [298]:
avg_loyalty_points = df['loyalty_accrual_rub_amt'].mean()
print(f"Среднее начисление бонусных баллов: {avg_loyalty_points:.2f} руб.")

Среднее начисление бонусных баллов: 644.11 руб.


In [299]:
df['created_dttm'] = pd.to_datetime(df['created_dttm'])
df['cancel_dttm'] = pd.to_datetime(df['cancel_dttm'])

df['booking_year'] = df['created_dttm'].dt.year
df['booking_month'] = df['created_dttm'].dt.month_name()

In [300]:
df['stay_duration_days'] = (pd.to_datetime(df['book_end_dttm']) - pd.to_datetime(df['book_start_dttm'])).dt.days
avg_stay_duration = df['stay_duration_days'].mean()
print(f"\nСредняя продолжительность бронирования: {avg_stay_duration:.2f} дней")


Средняя продолжительность бронирования: 2.16 дней


Преобладает краткосрочное бронирование (на 2 дня), что позволяет предположить, что отели чаще всего бронируют для деловых поездок.

In [301]:
most_popular_month = df['booking_month'].mode()[0]
print(f"Наиболее популярный месяц для бронирования: {most_popular_month}")

Наиболее популярный месяц для бронирования: July


Наиболее популярным месяцем для бронирования является Июль. Этот месяц соответсвует сезону отпусков.  

In [302]:
df['party_first_order_dt'] = pd.to_datetime(df['party_first_order_dt'])
account_lifespan = (df['created_dttm'].max() - df['party_first_order_dt'].min()).days
print(f"Максимальная длительность существования аккаунта: {account_lifespan} дней")

Максимальная длительность существования аккаунта: 2599 дней


Самый длительный аккаунт был создан человеком, который пользуется им более 7 лет. 

In [303]:
if 'promo_code_discount_amt' in df.columns:
    promo_usage = df['promo_code_discount_amt'].notna()
    print(f"Доля заказов с промокодами: {promo_usage.mean()*100:.2f}%")
    
if 'order_type_cd' in df.columns:
        print("\nИспользование промокодов по типам заказов:")
        print((df.groupby('order_type_cd')['promo_code_discount_amt'].apply(lambda x: x.notna().mean()*100)).round(2))

Доля заказов с промокодами: 73.68%

Использование промокодов по типам заказов:
order_type_cd
AIR    100.00
HOT      0.33
Name: promo_code_discount_amt, dtype: float64


In [ ]:
if {'promo_code_discount_amt', 'nominal_price_rub_amt'}.issubset(airport_df.columns):    promo_orders1 = airport_df[airport_df['promo_code_discount_amt'].notna()]   
organic_orders1 = airport_df[airport_df['promo_code_discount_amt'].isna()]

print(f"\nСредний чек с промокодом: {promo_orders1['nominal_price_rub_amt'].mean():.0f} руб.")
print(f"Средний чек без промокода: {organic_orders1['nominal_price_rub_amt'].mean():.0f} руб.")




Средний чек с промокодом: 16073 руб.
Средний чек без промокода: nan руб.


Средний чек покупки авиабилетов с промокодом равен 16073 руб. Без промокода данные отсутствуют, так как покупки всего совершаются с промокодом. 

In [355]:
if {'promo_code_discount_amt', 'nominal_price_rub_amt'}.issubset(hotels_df.columns):    promo_orders = hotels_df[hotels_df['promo_code_discount_amt'].notna()]   
organic_orders = hotels_df[hotels_df['promo_code_discount_amt'].isna()]

print(f"\nСредний чек с промокодом: {promo_orders['nominal_price_rub_amt'].mean():.0f} руб.")
print(f"Средний чек без промокода: {organic_orders['nominal_price_rub_amt'].mean():.0f} руб.")




Средний чек с промокодом: 10699 руб.
Средний чек без промокода: 11973 руб.


Средний чек бронирования отелей с промокодом выше на 1274 руб.

In [305]:
com_flags = {
    'suppress_email_flg': 'Отказ от email',
    'suppress_call_flg': 'Отказ от звонков'
}
for col, label in com_flags.items():
    if col in df.columns:
        opt_out = df[col].mean()*100
        print(f"{label}: {opt_out:.1f}% клиентов")

Отказ от email: 1.1% клиентов
Отказ от звонков: 3.2% клиентов


Процент отказов довольно низкий. 

In [306]:
if 'last_sms_success_flg' in df.columns:
    sms_success = df['last_sms_success_flg'].mean()*100
    print(f"\nУспешность последнего SMS: {sms_success:.2f}%")



Успешность последнего SMS: 83.55%


In [307]:
if 'bounce_cd' in df.columns:
    email_failure = df['bounce_cd'].notna().mean()*100
    print(f"Проблемы с доставкой email: {email_failure:.2f}%")
    

Проблемы с доставкой email: 4.75%


In [308]:
contact_cols = {
    'call_contact_6m_flg': 'Контакт (6 мес)',
    'call_contact_3m_flg': 'Контакт (3 мес)',
    'call_contact_1m_flg': 'Контакт (1 мес)'
}

for col, label in contact_cols.items():
    if col in df.columns:
        contact_rate = df[col].mean()*100
        print(f"{label}: {contact_rate:.2f}% клиентов")

Контакт (6 мес): 0.00% клиентов
Контакт (3 мес): 0.00% клиентов
Контакт (1 мес): 0.00% клиентов


In [309]:
loyalty_prog_dist = df['loyalty_program_type_nm'].value_counts().head(10)
print("\nРаспространённость программ лояльности:")
print(loyalty_prog_dist)


Распространённость программ лояльности:
loyalty_program_type_nm
Tinkoff Black                     289081
Tinkoff Black Premium             189005
All Airlines                      143114
Bravo                              41428
All Airlines Premium Кредитный     28634
All Airlines Premium               16277
AllAirLinesDebit                   14824
Tinkoff Black Private               6994
Tinkoff Platinum                    6154
Tinkoff Platinum Premium            5737
Name: count, dtype: int64


In [310]:
total_records = loyalty_prog_dist.sum()
loyalty_prog_percentage = ((loyalty_prog_dist / total_records) * 100).round(2)

print("\nРаспространенность программ лояльности (в процентах):")
print(loyalty_prog_percentage.apply(lambda x: f'{x:.2f}%'))


Распространенность программ лояльности (в процентах):
loyalty_program_type_nm
Tinkoff Black                     39.00%
Tinkoff Black Premium             25.50%
All Airlines                      19.31%
Bravo                              5.59%
All Airlines Premium Кредитный     3.86%
All Airlines Premium               2.20%
AllAirLinesDebit                   2.00%
Tinkoff Black Private              0.94%
Tinkoff Platinum                   0.83%
Tinkoff Platinum Premium           0.77%
Name: count, dtype: object


Продукты линейки Tinkoff приобладают (Tinloff Black 39.00%, Tinkoff Black Premium 25.50%), на третем месте находится All Airlines (19.31%). 

In [311]:
subscription_usage = df['bundle_nm'].value_counts()
print("\nИспользование подписок:")
print(subscription_usage)


Использование подписок:
bundle_nm
Premium            292078
Pro                286277
Private              8364
Team Premium          824
Team Select           271
Team Pro              150
Team Private           37
Tinkoff Premium        25
Tinkoff Pro            23
Name: count, dtype: int64


In [312]:
total_subscriptions = subscription_usage.sum()
subscription_percentage = ((subscription_usage / total_subscriptions) * 100).round(2)

print("\nИспользование подписок (в процентах):")
print(subscription_percentage.apply(lambda x: f'{x:.2f}%'))


Использование подписок (в процентах):
bundle_nm
Premium            49.67%
Pro                48.68%
Private             1.42%
Team Premium        0.14%
Team Select         0.05%
Team Pro            0.03%
Team Private        0.01%
Tinkoff Premium     0.00%
Tinkoff Pro         0.00%
Name: count, dtype: object


Большинство пользователей используют Premium (49.67%) и Pro (48.68%)подписки.